In [1]:
import requests
import requests_cache
from bs4 import BeautifulSoup
import json
import time
import pandas as pd
import datetime as dt
import re
from selenium import webdriver
import urllib.robotparser

In [2]:
urls = ['https://tiki.vn/api/v2/products?limit=48&&category=1846&page={}&urlKey=laptop-may-vi-tinh-linh-kien',
        'https://tiki.vn/api/v2/products?limit=48&&category=1789&page={}&urlKey=dien-thoai-may-tinh-bang',
        'https://tiki.vn/api/v2/products?limit=48&&category=4384&page={}&urlKey=bach-hoa-online']

In [3]:
rp = urllib.robotparser.RobotFileParser()
rp.set_url('https://tiki.vn/robots.txt')
rp.read()
print(rp.can_fetch('*', urls[0]))
print(rp.can_fetch('*', urls[1]))
print(rp.can_fetch('*', urls[2]))

True
True
True


In [4]:
sleep_time = 1
headers = { 'X-Requested-With': 'XMLHttpRequest', 
            "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36"}

In [5]:
def collect_products(urls, products_file):     
    f = open(products_file,'w', encoding = 'utf-8')
    f.write('id\tname\tbrand\tdiscount_price\tprice\trating\tnum_reviews\tquantity_sold\n')   
    
    for url in urls:
        i=1
        while (True):
            requests_cache.install_cache()
            r = requests.get(url.format(i), headers=headers)
            if not r.from_cache: 
                time.sleep(sleep_time)
                r = requests.get(url.format(i), headers=headers)

            if (r.status_code != 200):
                break
            items = json.loads(r.text)["data"]

            if (len(items) == 0):
                break

            for item in items:
                f.write(str(item['id']) +'\t')
            
                f.write(item['name'].replace('\t','')+'\t')                
                try:
                    f.write(item['brand_name'].replace('\t','')+'\t')
                except:
                    f.write("None" +'\t')
                f.write(str(item['price'])+'\t')
                f.write(str(item['list_price'])+'\t')
                f.write(str(item['rating_average'])+'\t')
                f.write(str(item['review_count'])+'\t')
                if 'quantity_sold' in item:
                    f.write(str(item['quantity_sold']['value'])+'\n')
                else:
                    f.write('0'+'\n')
            i += 1
           
    f.close() 

In [6]:
collect_products(urls, 'products.tsv')

In [7]:
df = pd.read_csv('products.tsv', sep='\t')

In [8]:
df

,id,name,brand,discount_price,price,rating,num_reviews,quantity_sold
0,76181490.0,Phần Mềm Diệt Virus BKAV Profressional 12 Thán...,Bkav,195000.0,299000.0,4.8,927.0,23907.0
1,416978.0,Bộ Chuyển Đổi USB Wifi Nano TP-Link TL-WN725N ...,TP-Link,117300.0,239000.0,4.5,611.0,6677.0
2,53056161.0,Phần mềm Microsoft 365 Family English APAC EM ...,Microsoft,1190000.0,1690000.0,5.0,720.0,3132.0
3,73795655.0,Apple Macbook Pro 2020 M1 - 13 Inchs (Apple M1...,Apple,30990000.0,34990000.0,5.0,209.0,460.0
4,23393172.0,Máy in HP Neverstop Laser 1000w (In/Wifi/Trắn...,HP,3100000.0,4290000.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
20016,97251112.0,Bột Cà Chua Trivie,Trivie,84000.0,84000.0,0.0,0.0,1.0
20017,85095152.0,LỌC THÁC VIPSUN 303H - LỌC THÁC MINI CHO HỒ CÁ...,OEM,100000.0,104000.0,0.0,0.0,1.0
20018,791327.0,Hỗn Hợp Ngũ Cốc Roasted Muesli Brookfarm (350g),BROOKFARM,236000.0,236000.0,3.5,6.0,244.0
20019,112086010.0,Cháo Ăn Liền Cao Cấp Khánh Hòa Nutrition,Khánh Hòa Nutrition,220000.0,220000.0,0.0,0.0,2.0
